In [2]:
from helper_fncts.GMM_Est_Fncts import *
import pandas as pd

# load clean_data/analysis_data_filtered.csv
analysis_data = pd.read_csv('../clean_data/analysis_data_filtered.csv')
analysis_data.head()
# define df for estimate_elasticities_gmm
# columns: 'i', 'j', 't' (all from analysis_data) 'y_ijt' = delta_a_ijdt, 'p_jt' = delta_p_jdt, 's_ijt' = delta_s_ijdt
df = pd.DataFrame(
    {
        'i': analysis_data['i'],
        'j': analysis_data['j'],
        't': analysis_data['t'],
        'y_ijt': analysis_data['delta_a_ijdt'],
        'p_jt': analysis_data['delta_p_jdt'],
        's_ijt': analysis_data['delta_s_ijdt'],
        'tradeable': analysis_data['tradeable']
    }
)

theta_i, gamma_j = estimate_elasticities_gmm(df, gamma_type="j-specific", theta_type="i-specific")
commongamma_theta_i, commongamma_gamma = estimate_elasticities_gmm(df, gamma_type="common", theta_type="i-specific")
commontheta_theta, commontheta_gamma = estimate_elasticities_gmm(df, gamma_type="common", theta_type="common")
nogamma_theta_i, nogamma_gamma = estimate_elasticities_gmm(df, gamma_type="none", theta_type="i-specific")
nogamma_theta, nogamma_gamma = estimate_elasticities_gmm(df, gamma_type="none", theta_type="common")


--- DATA DIAGNOSTICS (After Demeaning) ---
y_demeaned: std=0.169951, mean=-1.411824e-19
p_demeaned: std=0.054885, mean=1.411824e-20
s_demeaned: std=0.011659, mean=1.588302e-20

--- GMM SETUP ---
Number of parameters to estimate: 75 (theta: 66, gamma: 9)
Number of moment conditions: 132 (price moments: 66; share moments: 66)
Number of observations: 31455

Estimating GMM with identity weighting (W = I)...
Sum of squared moments: 8.14e-09
Max abs moment: 4.64e-05

Computing standard errors...
Computing Jacobian of moment conditions...

Theta estimates:
    i  theta_i  se_robust   t_stat
111CA 0.351777   0.114288 3.077988
113FF 0.000056   0.166628 0.000335
  211 0.000058   0.096416 0.000600
  212 0.198951   0.099644 1.996624
  213 0.431179   0.101212 4.260167
   22 0.247715   0.102307 2.421295
   23 0.425561   0.069580 6.116167
311FT 0.000060   0.217124 0.000275
313TT 0.604947   0.214963 2.814191
315AL 1.188743   0.131423 9.045165
... (56 more)

Gamma estimates:
     j   gamma_j    se_rob

In [ ]:
# combine theta results into single df
# repeat common theta 
theta_results = pd.concat([
    theta_i.assign(model='i-theta_j-gamma'),
    commontheta_theta.assign(model='uniform'),
    commongamma_theta_i.assign(model='commongamma'),
    nogamma_theta_i.assign(model='nogamma_i-specific'),
    nogamma_theta.assign(model='nogamma_common')
])

gamma_results = pd.concat([
    gamma_j.assign(model='i-theta_j-gamma'),
    commontheta_gamma.assign(model='commontheta'),
    commongamma_gamma.assign(model='commongamma')
])

theta_results.to_csv('../clean_data/theta_estimates.csv', index=False)
gamma_results.to_csv('../clean_data/gamma_estimates.csv', index=False)